In [170]:
#### import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd
import datetime
import json
import yaml
import requests
import bs4
import copy
import glob
from collections import OrderedDict
from copy import deepcopy

import imp

utils = imp.load_source('utils','../utils.py')

%matplotlib inline  
%pylab inline
pylab.rcParams['figure.figsize'] = (20,12)


Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning:

pylab import has clobbered these variables: ['text', 'copy']
`%matplotlib` prevents importing * from pylab and numpy



#### CONECTA AO BANCO

In [47]:
with open('server_config.yaml', 'r') as f:
        server = yaml.load(f)

host = server['host']
database = server['database']
user = server['user']
password = server['password']


url = 'postgresql://{}:{}@{}/{}'
url = url.format(user, password, host, database)
conn = sqlalchemy.create_engine(url)

##### CONECTA PLOTLY

In [48]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username=server['plotly_username_1'], api_key=server['plotly_key_1'])
#plotly.tools.set_credentials_file(username=server['plotly_username_2'], api_key=server['plotly_key_2'])

#### INFOS GERAIS

In [49]:
with open('../partidos_cor.yaml', 'r') as f:
        partidos_todos = yaml.load(f)

In [166]:
# IMPORT DATA
dados = {'PL':{}, 'MP':{}, 'PEC':{}, 'todos':{}}
for path in glob.glob('../indicadores/csvs/camdep_apoio_governo_bancada_voto_parlamentar_dema_*'):

    tipo = path.split('dema_')[1].split('_')[0]
    hl = int(path.split('dema_')[1].split('_')[1].split('.')[0])
    dados[tipo][hl] = pd.read_csv(path)

In [167]:
# ORDER DATA
dados_i = {}
for k, d in dados.items():
    dados_i[k] = OrderedDict(sorted(d.items(), key=lambda x: x[0]))
    
dados = dados_i

#### MÉDIA DOS DEPUTADOS

In [24]:
fidelidade_deputado = pd.read_sql_query('SELECT * FROM desenv.votacao_governo_deputados_dia ORDER BY datavotacao DESC',
                                        conn)
votacao_id_table = pd.read_sql_query('SELECT * FROM desenv.id_proposicao_com_orient_governo',
                                        conn)
proposicoes = pd.read_sql_query('SELECT * FROM a_camdep.votacao_bancada_partidos',
                                conn)

In [25]:
fidelidade_deputado.head(1)

,index,codproposicao_x,tipo_x,numero,ano,codsessao,datavotacao,idecadastro,nome,partido,uf,voto,orientacao,votacao_id,fidelidade,quorum,partido_atualizado,tam_bancada
0,0,2127233,MPV,770,2017,16252,2017-06-28 12:22:00,74784.0,Luiza Erundina,PSOL,SP,Obstrução,Não,4030020129133669703,0,386,PSOL,6


In [53]:
partido = 'PSDB'

In [102]:
bancada = list(set(fidelidade_deputado[fidelidade_deputado['partido'] == partido]['idecadastro'].tolist()))

In [153]:
bancada_apoio = fidelidade_deputado.pivot(index='votacao_id', columns='idecadastro', values='fidelidade')[bancada]
bancada_apoio = bancada_apoio.reset_index().merge(votacao_id_table, on='votacao_id').set_index('datavotacao').sort_index()
bancada_apoio = bancada_apoio.drop(['votacao_id', 'codsessao', 'codproposicao'], axis=1)

In [117]:
bancada_apoio.head(1)

,166401.0,74753.0,159238.0,164359.0,74758.0,74252.0,74768.0,74771.0,74262.0,74777.0,...,73693.0,74205.0,74209.0,74211.0,74212.0,74216.0,74221.0,160758.0,74235.0,74748.0
datavotacao,,,,,,,,,,,,,,,,,,,,,
1998-11-04 17:46:00,NaN,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,...,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0


In [108]:
# para cada parlamentar intervalos de tempo no partido
partido_interval = {}
for parlamentar in bancada:
    parl = fidelidade_deputado[fidelidade_deputado['idecadastro'] == parlamentar]
    date = parl[parl['partido'] == partido]['datavotacao']
    partido_interval[parlamentar] = (date.min(), date.max())

In [156]:
# usar partido_interval como máscara para bancada_apoio
def mask(parlamentar):
    
    mini, maxi = partido_interval[parlamentar.name]
    
    null = pd.Series([np.nan for i in range(len(parlamentar))], index=parlamentar.index)
    null[mini: maxi] = parlamentar[mini: maxi]    
    
    return null

bancada_apoio = bancada_apoio.apply(mask)

In [160]:
bancada_apoio_dema = utils.calcula_medias_dema({partido: bancada_apoio})

In [162]:
bancada_apoio_dema = utils.normaliza(bancada_apoio_dema)

In [168]:
for dias in dados['todos'].keys():
    dados['todos'][dias] = dados['todos'][dias].set_index('datavotacao')
    dados['todos'][dias] = dados['todos'][dias].merge(bancada_apoio_dema[dias][partido], left_index=True, right_index=True)

In [172]:
# PLOT
range_y0, range_y1 = 0, 100

def rounde(x):
    if x != None:
        return round(x, 2)
    else:
        return x
    
data = []
for tipo, halflifes in dados.items():
    if tipo == 'todos':
        for halflife, fidelidade_deputado_partido in halflifes.items():

            #fidelidade_deputado_partido['datavotacao'] = fidelidade_deputado_partido['datavotacao'].apply(
            #    lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
            #fidelidade_deputado_partido.set_index('datavotacao', inplace=True)
            #idelidade_deputado_partido.drop('Apoio Médio.1', axis=1, inplace=True)
            for partido in fidelidade_deputado_partido.columns:
                
                
                visible = False
                if (halflife == 90):
                    visible = 'legendonly'
                    if (partido == 'Apoio Médio'):
                        visible = True
                
                if partido in partidos_todos.keys():        
                    d = go.Scatter(
                            visible = visible,
                            name = '{0}'.format(partido),
                            mode = 'lines',
                            line = dict(
                                        color = (partidos_todos[partido]['cor']),
                                        width = 2),
                            x = fidelidade_deputado_partido.index,
                            y = fidelidade_deputado_partido[partido].apply(rounde),
                            hoverinfo='y+text',
                            text='% de apoio',
                            yaxis='y1'
                        )
                else:
                    d = go.Scatter(
                            visible = visible,
                            name = 'Sérgio Zvaiter',
                            mode = 'lines',
                            line = dict(
                                        color = 'black',
                                        width = 2),
                            x = fidelidade_deputado_partido.index,
                            y = fidelidade_deputado_partido[partido].apply(rounde),
                            hoverinfo='y+text',
                            text='% de apoio',
                            yaxis='y1'
                        )

                data.append(d)


votacoes_dia_e = dados['todos'][7].resample('M').count()['PP']
data.append(go.Bar(
            name='Número de Votações',
            x=votacoes_dia_e.index,
            y=votacoes_dia_e,
            marker=dict(color='grey'),
            text='Votações no Mês',
            hoverinfo='y+text',
            yaxis='y2', 
            opacity=0.4,
            ))
           
### RETANGULOS PRESIDENTES
rectangle_color = '#E2E2E6'
opacity = 1
presidentes = [
        # LULA I
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2003-01-01','y0': range_y0,'x1': '2006-12-31','y1': range_y1,
            'fillcolor':rectangle_color ,'opacity': opacity ,'line': {
                'width': 0
            }, 'layer':'below',
        },
        # DILMA I
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2011-01-01','y0': range_y0,'x1': '2014-12-31','y1': range_y1,
            'fillcolor': rectangle_color,'opacity': opacity ,'line': {
                'width': 0
            },'layer':'below'
        },
        # TEMER 
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2016-05-01','y0': range_y0,
            'x1': datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], '%Y-%m-%d'),
            'y1': range_y1,
            'fillcolor': rectangle_color,'opacity': opacity ,'line': {
                'width': 0,
            }, 'layer':'below'
        }]  

### ANOTACOES PRESITENTES
y_pred = 1.05
presidentes_hist = [dict(x='2000-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='FHC II',
                      ax=0, ay=-30,
                      showarrow=False,
                        font=dict(size=9)),
                    dict(x='2004-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='LULA I',
                      ax=0, ay=-30,
                      showarrow=False,
                        font=dict(size=9)),
                   dict(x='2008-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='LULA II',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2012-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='DILMA I',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2015-08-15',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='DILMA II',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2016-12-01',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='TEMER',
                      ax=0, ay=-30,
                      showarrow=False,
                      font=dict(size=9))]

#### RETANGULOS PRESIDENTE CAMARA
rectangle_color = '#E2E2E6'
opacity = 1
shape_skeleton = {'type': 'rect','xref': 'x','yref': 'y','x0': '2003-01-01','y0': range_y0,'x1': '2006-12-31',
                  'y1': range_y1,'fillcolor':rectangle_color ,'opacity': opacity ,'line': {'width': 0}, 'layer':'below'}


presidentes_camara = [ {'mandato': ('1999-02-02','2001-02-13'), 'nome': 'TEMER', 'partido':'PMDB'},
                       {'mandato': ('2001-02-13','2002-12-17'), 'nome': 'AÉCIO', 'partido':'PSDB'},
                       {'mandato': ('2003-02-02','2005-02-14'), 'nome': 'LIMA'  , 'partido':'PT'},
                       {'mandato': ('2005-02-14','2005-07-21'), 'nome': '*', 'partido':''},
                       {'mandato': ('2005-07-28','2007-02-01'), 'nome': 'REBELO', 'partido':'PCdoB'},
                       {'mandato': ('2007-02-01','2009-02-02'), 'nome': 'CHINAGLIA' , 'partido':'PT'},
                       {'mandato': ('2009-02-02','2010-12-17'), 'nome': 'TEMER', 'partido':'PMDB'  },
                       {'mandato': ('2010-12-17','2013-02-04'), 'nome': 'AURELI', 'partido':'PT'},
                       {'mandato': ('2013-02-04','2015-02-01'), 'nome': 'ALVES', 'partido':'PMDB'},
                       {'mandato': ('2015-02-01','2016-07-07'), 'nome': 'CUNHA', 'partido':'PMDB'},
                       {'mandato': ('2016-07-07',
                                   datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], '%Y-%m-%d')),
                      'nome': 'MAIA', 'partido':'DEM'}]


presidentes_camara_shapes = []
for i in range(len(presidentes_camara)):
    if i % 2 != 0:
        shape_skeleton.update(dict(zip(['x0','x1'],presidentes_camara[i]['mandato'])))
        presidentes_camara_shapes.append(deepcopy(shape_skeleton))

#### ANOTACOES PRESIDETES CAMARA
y_pred = 1.05
annot_skeleton = dict(x='2000-12-31', y=y_pred, xref='x', yref='paper', text='FHC II',ax=0, ay=-30,
                     showarrow=False,
                       font=dict(size=9))

presidentes_camara_annot = []
for presidente in presidentes_camara:
    a, b  = presidente['mandato']
    a = datetime.datetime.strptime(a, '%Y-%m-%d')
    b = datetime.datetime.strptime(b, '%Y-%m-%d')
    mean = a + (b - a) /2
    annot_skeleton.update(dict(zip(['x','text'], (mean, '{0}<br>{1}'.format(presidente['nome'], presidente['partido'])))))
    presidentes_camara_annot.append(deepcopy(annot_skeleton))
    
severino = [dict(text='* SEVERINO PP', font=dict(size=9), x=1.12, y=-0.4, yref='paper', xref='paper', showarrow=False)]
    


### ACONTECIMENTOS
acontecimentos = [dict(x='2013-06-23',
                      y=1,
                      xref='x', yref='paper',
                      text='Jornadas de <br> Junho',
                      ax=0, ay=-18,
                      font=dict(size=10)),
               dict(x='2005-06-30',
                    y=1,
                    xref='x', yref='paper',
                    text='Mensalão',
                    ax=0, ay=-12,
                   font=dict(size=10)),
                dict(x='2008-09-15',
                    y=1,
                    xref='x', yref='paper',
                    text='Crise <br> Financeira',
                    ax=0, ay=-18,
                    font=dict(size=10)),
                dict(x='2016-05-12',
                    y=1,
                    xref='x', yref='paper',
                    text='Impeachment',
                    ax=0, ay=-12,
                    font=dict(size=10)),]
                    

acontecimentos_lines =  [{
                            'type': 'line',
                            'x0': ac['x'],
                            'y0': range_y0,
                            'x1': ac['x'],
                            'y1': range_y1,
                            'line': {
                                'color': 'black',
                                'width': 1,
                            },
                        } for ac in acontecimentos]


# LINHAS DO GRAFICO
shapes_inidicator_color = '#000000'
shapes_indicator = [dict(visible=True,
                        layer='above',
                        line=dict(color=shapes_inidicator_color,
                                 width=1,
                                 dash='dot'),
                        type='line',
                        xref='paper',
                        x0=-0.005,
                        x1=1,
                        yref='y',
                        y0=50,
                        y1=50),
                   dict(visible=True,
                        layer='above',
                        line=dict(color=shapes_inidicator_color,
                                 width=1,
                                 dash='dot'),
                        type='line',
                        xref='paper',
                        x0=-0.005,
                        x1=1,
                        yref='y',
                        y0=100,
                        y1=100),
                   dict(visible=True,
                        layer='above',
                        line=dict(color=shapes_inidicator_color,
                                 width=1,
                                 dash='solid'),
                        type='line',
                        xref='paper',
                        x0=-0.006,
                        x1=1,
                        yref='y',
                        y0=0,
                        y1=0)]


def make_visible(where_true, data):
    """ Function that produces a list of true or false depending on visibility
    :param where_true: tuple (tipo, halflife)
    :param data: list of data
    :return: list of True and False
    """
    #length = int(len(data) / 4 / 3)
    data, bar = data[:-1], data[-1]
    length = int(len(data) / 3)
    tipos = {'todos' : 0, 'PL' : 3, 'MP' : 6, 'PEC' : 9}
    halflifes = {7 : 0, 30 : 1, 90 : 2}
   
    #position = [ 0  for i in range(12)]
    position = [ 0  for i in range(3)]
    position[tipos[where_true[0]] + halflifes[where_true[1]]] = 1
    

    visible = []
    for t in position:
        if t == 1:
            visible = visible + [True]
            visible = visible + ['legendonly' for i in range(length-1)]
        else:
            visible = visible + [False for i in range(length)]
    
    visible = visible + [True]
    return visible


### TEXTO DO GRAFICO

subtitulos = [#dict(text='PRESIDENTES', font=dict(size=11), x=1.1, y=0.86, yref='paper', xref='paper',
              #     showarrow=False, xanchor='center'),
             #dict(text='INFORMAÇÕES', font=dict(size=11), x=1.1, y=0.72, yref='paper', xref='paper', 
              #    showarrow=False, xanchor='center'),
              #dict(text='MÉDIA', font=dict(size=11), x=1.1, y=1, yref='paper', xref='paper',
               #    showarrow=False, xanchor='center'),
              #dict(text='ESPÉCIE LEGISLATIVA', font=dict(size=11), x=0, y=-0.55, yref='paper', xref='paper', showarrow=False),
             # dict(text='PARTIDOS', font=dict(size=11), x=0, y=-0.1, yref='paper', xref='paper', showarrow=False),
             dict(text='<b>ENTENDA</b><br>Com as votações na Câmara<br>é possível calcular o apoio<br>dos partidos ao governo.<br><br>O calculo leva em conta a <br>proporção média de <br>deputados de cada <br>bancada que votou <br>segundo a orientação <br>do governo em plenário.', 
                  font=dict(size=12),
                  align='left',
                  xref='paper',
                  x=1.23,
                  xanchor='auto',
                  yref='paper',
                  y=1.02,
                  showarrow=False),
             dict(text='<b>EXPLORE</b><br>Para começar basta escolher<br>uma <b>Média</b>.<br><br><i>7 dias</i> mostra como foi<br>o apoio em uma semana.<br><br><i>30 dias</i> mostra a tendência<br>do apoio em um mês.<br><br><i>90 dias </i>pode ser usado<br>para enteder a dinâmica<br>do partido no longo<br>prazo.', 
                  font=dict(size=12),
                  align='left',
                  xref='paper',
                  x=1.23,
                  xanchor='auto',
                  yref='paper',
                  y=0.48,
                  showarrow=False),]
"""
             dict(text='<b>SAIBA MAIS</b><br>Entenda a <a href=\"\">matemática</a><br>por trás do gráfico.<br><br>O <a href=\"\">código</a> é aberto,<br>faça sua contribuição.<br><br><a href=\"\">Dados</a> da Câmara dos<br>Deputados.<br><br>Pode ficar melhor?<br>Entre em <a href=\"\">contato</a>.', 
                  font=dict(size=12),
                  align='left',
                  xref='paper',
                  x=1.02,
                  xanchor='left',
                  yref='paper',
                  y=-0.37,
                  yanchor='auto',
                  showarrow=False,),]

dict(text='Atualizado: '.format(datetime.datetime.strftime(datetime.datetime.now(), '%d/%m/%Y')),
font=dict(size=12),
align='center',
xref='paper',
x=1.15,
xanchor='left',
yref='paper',
y=-0.41,
yanchor='auto',
showarrow=False)"""

#### UPDATE MENUS
updatemenus = list([
    dict(buttons=list([   
            dict(label = 'FHC II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=[datetime.datetime.strftime(fidelidade_deputado_partido.index[0], 
                                                                          '%Y-%m-%d'), 
                                               '2002-12-31'])}]),  
           
            dict(label = 'Lula I',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2003-01-01', '2006-12-31'])}]),

            dict(label = 'Lula II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2007-01-01', '2010-12-31'])}]),

            dict(label = 'Dilma I',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2011-01-01', '2014-12-31'])}]),

            dict(label = 'Dilma II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2015-01-01', '2016-05-01'])}]),
        
            dict(label = 'Temer',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2016-05-01', 
                                               datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], 
                                                                          '%Y-%m-%d')])}]),
             dict(label = 'Presidentes',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=[datetime.datetime.strftime(fidelidade_deputado_partido.index[0], 
                                                                          '%Y-%m-%d'), 
                                               datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], 
                                                                          '%Y-%m-%d')])}]),
            ]),
            direction = 'down',
            pad = {'r': 10, 't': 10},
            showactive = True,
            active = 6,
            type = 'dropdown',
            x = 1.02,
            xanchor = 'left',
            y = 0.1,
            yanchor = 'bottom' ),

    ## BOTOES INFORMACOES
    dict(buttons=list([   
        dict(label = 'Acontecimentos',
             method = 'relayout',
             args = [{'annotations': acontecimentos +  subtitulos,
                     'shapes': acontecimentos_lines + shapes_indicator,
                     'xaxis': dict(tickvals=[x['x'] for x in acontecimentos])}]),
        dict(label = 'Presidentes',
             method = 'relayout',
             args = [{'annotations': presidentes_hist + subtitulos,
                     'shapes': presidentes + shapes_indicator,
                     'xaxis': dict(tickvals=['2003', '2007', '2011', '2015', '2016-05'], tickformat='%Y')}]),
        dict(label = 'Presidentes Câmara',
             method = 'relayout',
             args = [{'annotations': presidentes_camara_annot + subtitulos + severino,
                     'shapes': presidentes_camara_shapes + shapes_indicator,
                     'xaxis': dict(tickvals=['2001','2003','2005', '2007','2009', '2011', '2013', '2015', '2007', '2017'], tickformat='%Y')}]),
         dict(label = 'Informações',
             method = 'relayout',
             args = [{'annotations': subtitulos,
                     'shapes': shapes_indicator,
                      'xaxis': dict(tickvals=['2003', '2007', '2011', '2015'], tickformat='%Y')}]),
        ]),
            direction = 'down',
            pad = {'r': 10, 't': 10},
            showactive = True,
            active = 3,
            type = 'dropdown',
            x = 1.02,
            xanchor = 'left',
            y = 0.0,
            yanchor = 'bottom' ),
    
    ## BOTOES ANALISE
    dict(buttons=list([   
        dict(label = '7 Dias',
             method = 'update',
             args = [{'visible': make_visible(('todos', 7), data)}]),
        dict(label = '30 Dias',
             method = 'update',
             args = [{'visible': make_visible(('todos', 30), data)}]),
        dict(label = '90 Dias',
             method = 'update',
             args = [{'visible': make_visible(('todos', 90), data)}]),
        dict(label = 'Média',
             method = 'update',
             args = [{'visible': make_visible(('todos', 90), data)}]),
        ]),
            direction = 'down', 
            active = 3,
            pad = {'r': 10, 't': 10},
            showactive = True,
            type = 'dropdown',
            x = 1.02,
            xanchor = 'left',
            y = 0.2,
            yanchor = 'bottom' ),

])



legend = dict(orientation='h', x=0, y=-0.18, xanchor='center', yanchor='middle')

yaxis = go.YAxis(
    #title="Apoio ao Governo",
    range=[range_y0, range_y1],
    showgrid=False,
    showline=True,
    ticks="", 
    showticklabels=True,
    #mirror=True,
    linewidth=1,
    ticktext=['Menos<br>Apoio', 'Mais<br>Apoio'],
    tickvals=[range_y0+(range_y1 - range_y0)/20, range_y1]
)



layout = dict(title = 'Apoio ao Governo das Bancadas Partidárias por Voto <br> Parlamentar na Câmara ',
              xaxis = dict(#rangeslider=dict(),
                            type='date'),
              yaxis = yaxis,
              yaxis2= dict(
                            range=[0,200],
                            visible=False,
                            overlaying='y',
                            side='right'
                        ),
              autosize=False,
              width=992,
              height=950,
              margin=go.Margin(
                    l=50,
                    r=150,
                    t=110,
                    b=250,
                    pad=5
                ),
              paper_bgcolor='#F2F2F2',
              plot_bgcolor='#F2F2F2',
              updatemenus=updatemenus,
              annotations=subtitulos,
              legend=legend,
              font = dict(family='Lato, sans-serif'),
              shapes = shapes_indicator,
)

            
fig = dict(data = data, layout=layout)
py.iplot(fig, filename='Apoio Governo Bancadas Voto Parlamentar Comparacao')

PlotlyRequestError: 
<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>502 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>
